In [ ]:
import yaml
from omegaconf import OmegaConf
from yolo_ev.module.model_module import ModelModule
from yolo_ev.module.data_module import DataModule

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers


save_dir = './result'

yaml_file = "./config/param.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)

data = DataModule(config)
model = ModelModule(config)



In [ ]:
data.setup('fit')

In [ ]:
from torch.utils.data import DataLoader
dataset = data.valid_dataset
dataloader = DataLoader(dataset)


In [ ]:
data_iter = iter(dataloader)
img, target, info, id = data_iter._next_data()

In [ ]:
target

In [ ]:
import torch
output_postprocessed = [torch.tensor([[217.6200, 240.5400, 256.6100, 298.2900, 0.9 , 0.9,   39.0000],
         [  1.0000, 240.2400, 347.6300, 427.0000, 0.9 , 0.9,  60.0000],
         [388.6600,  69.9200, 498.0700, 347.5400, 0.9 , 0.9,   0.0000],
         [135.5700, 249.4300, 157.8900, 278.2200, 0.9 , 0.9,  43.0000],
         [ 31.2800, 344.0000,  99.4000, 384.8300, 0.9 , 0.9,  45.0000],
         [ 59.6300, 287.3600, 135.7000, 328.6600, 0.9 , 0.9,  45.0000],
         [  1.3600, 164.3300, 193.9200, 262.7000, 0.9 , 0.9,  69.0000],
         [  0.0000, 262.8100,  62.1600, 299.5800, 0.9 , 0.9,   0.0000],
         [119.4000, 272.5100, 144.2200, 306.7600, 0.9 , 0.9,  41.0000],
         [141.4700, 267.9100, 173.6600, 303.7700, 0.9 , 0.9,  41.0000],
         [155.9700, 168.9500, 182.0000, 186.0800, 0.9 , 0.9,  45.0000],
         [157.2000, 114.1500, 175.0600, 129.9700, 0.9 , 0.9,  45.0000],
         [ 98.7500, 304.7800, 109.5300, 310.3500, 0.9 , 0.9,  50.0000],
         [166.0300, 256.3600, 174.8500, 274.9400, 0.9 , 0.9,  44.0000],
         [ 86.4100, 293.9700, 110.3700, 305.1500, 0.9 , 0.9,  50.0000],
         [ 70.1400, 296.1600,  79.4200, 300.7400, 0.9 , 0.9,  50.0000],
         [  0.0000, 210.9000, 191.3600, 309.8800, 0.9 , 0.9,  69.0000],
         [ 96.6900, 297.0900, 104.5300, 301.9500, 0.9 , 0.9,  51.0000],
         [497.2500, 203.4000, 619.2600, 232.0100, 0.9 , 0.9,  71.0000]])]


In [ ]:
from yolo_ev.utils.eval.evaluation import to_coco_format, evaluation
from yolo_ev.module.data.dataset.coco.coco_classes import COCO_CLASSES

categories = [{"id": id+1 , "name": class_name, "supercategory": "none"}
                  for id, class_name in enumerate(COCO_CLASSES)]
dataset, result = to_coco_format(gts=target, detections=output_postprocessed, categories=categories)

In [ ]:
result

In [ ]:
dataset

In [ ]:

scores = evaluation(Gt=dataset, Dt=result, num_data=1)

In [ ]:
scores['AP']

In [ ]:
import torch

ckpt = torch.load("./weight/yolox_m.pth")
model.model.load_state_dict(ckpt['model'])